In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import gym

import math, random, time
import matplotlib.pyplot as plt

In [2]:
env = gym.make('CartPole-v0')

seed_value = 23
env.seed(seed_value)
torch.manual_seed(seed_value)
random.seed(seed_value)

In [3]:
# PARAMS

learning_rate = 0.01
num_episodes = 500
gamma = 1

hidden_layer = 64

egreedy = 0.9
egreedy_final = 0
egreedy_decay = 500

report_interval = 10
score_to_solve = 195

replay_mem_size = 50000
batch_size = 32

update_target_frequency = 100
clip_error = False

double_dqn = True

In [4]:
number_of_inputs = env.observation_space.shape[0]
number_of_outputs = env.action_space.n

def calculate_epsilon(steps_done):
    return egreedy_final + (egreedy - egreedy_final) * math.exp(-1 * steps_done / egreedy_decay)

In [5]:
class ExperienceReplay(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0
 
    def push(self, state, action, new_state, reward, done):
        transition = (state, action, new_state, reward, done)
        
        if self.position >= len(self.memory):
            self.memory.append(transition)
        else:
            self.memory[self.position] = transition
        
        self.position = ( self.position + 1 ) % self.capacity
        
        
    def sample(self, batch_size):
        return zip(*random.sample(self.memory, batch_size))
        
        
    def __len__(self):
        return len(self.memory)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(number_of_inputs, hidden_layer)
        self.linear2 = nn.Linear(hidden_layer, number_of_outputs)
        self.activation = nn.Tanh()
#         self.activation = nn.ReLU()
        
    def forward(self, x):
        output1 = self.linear1(x)
        output1 = self.activation(output1)
        return self.linear2(output1)
        

In [7]:
class QNetAgent(object):
    def __init__(self):
        self.nn = NeuralNetwork()
        self.target_nn = NeuralNetwork()
        self.loss_func = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
        self.update_target_counter = 0
        
    def select_action(self, state, epsion):
        random_for_egreedy = torch.rand(1)[0]
        
        if random_for_egreedy > epsilon:
            with torch.no_grad():
                state = torch.Tensor(state)
                action_from_nn = self.nn(state)
                action = torch.max(action_from_nn, 0)[1]
                action = action.item()
        else:
            action = env.action_space.sample()
            
        return action
    
    def optimize(self):
        if (len(memory) < batch_size):
            return
        
        state, action, new_state, reward, done = memory.sample(batch_size)
        
        state = torch.Tensor(state) #.to(device)
        new_state = torch.Tensor(new_state) #.to(device)
        reward = torch.Tensor(reward) #.to(device)
        action = torch.LongTensor(action) #.to(device)
        done = torch.Tensor(done) #.to(device)
        
        if double_dqn:
            new_state_indexes = self.nn(new_state).detach()
            max_new_state_indexes = torch.max(new_state_indexes, 1)[1]  
            
            new_state_values = self.target_nn(new_state).detach()
            max_new_state_values = new_state_values.gather(1, max_new_state_indexes.unsqueeze(1)).squeeze(1)
        else:
            new_state_values = self.target_nn(new_state).detach()
            max_new_state_values = torch.max(new_state_values, 1)[0]
            
        target_value = reward + ( 1 - done ) * gamma * max_new_state_values
  
        predicted_value = self.nn(state).gather(1, action.unsqueeze(1)).squeeze(1)
        
        loss = self.loss_func(predicted_value, target_value)
    
        self.optimizer.zero_grad()
        loss.backward()
        
        if clip_error:
            for param in self.nn.parameters():
                param.grad.data.clamp_(-1,1)

        self.optimizer.step()
        
        if self.update_target_counter % update_target_frequency == 0:
            self.target_nn.load_state_dict(self.nn.state_dict())
        
        self.update_target_counter += 1


*** Episode 0 ***                       
Av.reward: [last 10]: 1.20, [last 100]: 0.12, [all]: 12.00                       
epsilon: 0.88, frames_total: 12
Elapsed time:  00:00:00

*** Episode 10 ***                       
Av.reward: [last 10]: 20.00, [last 100]: 2.12, [all]: 19.27                       
epsilon: 0.59, frames_total: 212
Elapsed time:  00:00:00

*** Episode 20 ***                       
Av.reward: [last 10]: 52.60, [last 100]: 7.38, [all]: 35.14                       
epsilon: 0.21, frames_total: 738
Elapsed time:  00:00:00

*** Episode 30 ***                       
Av.reward: [last 10]: 189.10, [last 100]: 26.29, [all]: 84.81                       
epsilon: 0.00, frames_total: 2629
Elapsed time:  00:00:02

*** Episode 40 ***                       
Av.reward: [last 10]: 179.40, [last 100]: 44.23, [all]: 107.88                       
epsilon: 0.00, frames_total: 4423
Elapsed time:  00:00:04

*** Episode 50 ***                       
Av.reward: [last 10]: 189.70, [last 10


*** Episode 440 ***                       
Av.reward: [last 10]: 189.70, [last 100]: 175.52, [all]: 183.62                       
epsilon: 0.00, frames_total: 80975
Elapsed time:  00:01:19

*** Episode 450 ***                       
Av.reward: [last 10]: 196.10, [last 100]: 179.07, [all]: 183.89                       
epsilon: 0.00, frames_total: 82936
Elapsed time:  00:01:20

*** Episode 460 ***                       
Av.reward: [last 10]: 200.00, [last 100]: 180.70, [all]: 184.24                       
epsilon: 0.00, frames_total: 84936
Elapsed time:  00:01:22

*** Episode 470 ***                       
Av.reward: [last 10]: 200.00, [last 100]: 188.38, [all]: 184.58                       
epsilon: 0.00, frames_total: 86936
Elapsed time:  00:01:24

*** Episode 480 ***                       
Av.reward: [last 10]: 200.00, [last 100]: 193.84, [all]: 184.90                       
epsilon: 0.00, frames_total: 88936
Elapsed time:  00:01:26

*** Episode 490 ***                       
Av.rew

In [8]:
qnet_agent = QNetAgent()
memory = ExperienceReplay(replay_mem_size)

steps_total = []
frames_total = 0
solved_after = 0
solved = False

start_time = time.time()

for i_episode in range(num_episodes):
    
    state = env.reset()
    
    step = 0
    while True:
        
        step += 1
        frames_total += 1
        
        epsilon = calculate_epsilon(frames_total)
        
        action = qnet_agent.select_action(state, epsilon)
        
        new_state, reward, done, info = env.step(action)

        memory.push(state, action, new_state, reward, done)
        qnet_agent.optimize()
        
        state = new_state
        
        if done:
            steps_total.append(step)
            
            mean_reward_100 = sum(steps_total[-100:])/100
            
            if (mean_reward_100 > score_to_solve and solved == False):
                print("SOLVED! After %i episodes " % i_episode)
                solved_after = i_episode
                solved = True
            
            if (i_episode % report_interval == 0):
                
                
                
                print("\n*** Episode %i *** \
                      \nAv.reward: [last %i]: %.2f, [last 100]: %.2f, [all]: %.2f \
                      \nepsilon: %.2f, frames_total: %i" 
                  % 
                  ( i_episode,
                    report_interval,
                    sum(steps_total[-report_interval:])/report_interval,
                    mean_reward_100,
                    sum(steps_total)/len(steps_total),
                    epsilon,
                    frames_total
                          ) 
                  )
                  
                elapsed_time = time.time() - start_time
                print("Elapsed time: ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))



            break


print("Average reward: %.2f" % (sum(steps_total)/num_episodes))
print("Average number of steps (last 100 episodes): %.2f" % (sum(steps_total[-100:])/100))
if solved:
      print("Solved after %i episodes" % solved_after)
plt.figure(figsize=(12,5))
plt.title("Rewards")
plt.bar(torch.arange(len(steps_total)), steps_total, alpha=0.6, color='red')
plt.show()
      
env.close()
env.env.close()